# 데이터 전처리
1. 결측치
2. 이상치
3. 파생변수 생성
4. 원 핫 인코딩
5. 데이터 병합
6. 데이터 정렬 및 변환

## 1. 결측치

In [173]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split

raw_data = {
    "EMP_NO": [1, np.NaN, 3, 4, 5, 6],
    "NAME": [np.NaN, "Bill", "Charlie", "Dragon", "Entropy", "False"],
    "SALARY": [1000, 3000, 2000, 5000, 10000, 100],
    "DEPARTMENT": ["IT", "IT", "HR", "IT", np.NAN, "HR"]
}


df = pd.DataFrame(raw_data)

df.head(5)

,EMP_NO,NAME,SALARY,DEPARTMENT
0,1.0,NaN,1000,IT
1,NaN,Bill,3000,IT
2,3.0,Charlie,2000,HR
3,4.0,Dragon,5000,IT
4,5.0,Entropy,10000,NaN


In [78]:
# 이상치가 존재하는 컬럼 (하나라도 존재 = any(), 전부 이상치 = all())
ex1 = df.isna().any()

# 이상치가 존재하는 행 확인
ex2 = df.isna().any(axis=1)

# 이상치가 하나라도 존재하는 컬럼 제거
ex3 = df.dropna(axis=1, how='any')

# 특정 컬럼만 이상치 검사
ex4 = df.dropna(subset=["DEPARTMENT"])

,EMP_NO,NAME,SALARY,DEPARTMENT
0,1.0,Alien,1000,IT
1,NaN,Bill,3000,IT
2,3.0,Charlie,2000,HR
3,4.0,Dragon,5000,IT
5,6.0,False,100,HR


In [93]:
# 특정 컬럼을 지정해 결측치 채우기
df.fillna(
    {
        "EMP_NO": df["EMP_NO"].quantile(q=0.5)
    }
)

# 앞, 뒤 값으로 결측치 채우기
df.fillna(method='ffill').fillna(method='bfill')

,EMP_NO,NAME,SALARY,DEPARTMENT
0,1.0,Bill,1000,IT
1,1.0,Bill,3000,IT
2,3.0,Charlie,2000,HR
3,4.0,Dragon,5000,IT
4,5.0,Entropy,10000,IT
5,6.0,False,100,HR


## 2. 이상치

In [107]:
# p 95 이상인 값 제거
threshold = df["SALARY"].quantile(q=0.95)
df[df["SALARY"] < threshold]

,EMP_NO,NAME,SALARY,DEPARTMENT
0,1.0,NaN,1000,IT
1,NaN,Bill,3000,IT
2,3.0,Charlie,2000,HR
3,4.0,Dragon,5000,IT
5,6.0,False,100,HR


## 3. 파생 변수 생성

In [132]:
# 연봉이 5000 이상이면 rich, 아니면 commoner
df["rank"] = np.select([df["SALARY"] >= 5000], ["rich"], default="commoner")
df

# 연봉 +5000-1000/10 가 800 이상이면 rich, 아니면 commoner
df["rank"] = np.select(
    [df["SALARY"].apply(func=lambda x: (x+5000-1000)/10) >= 800], 
    ["rich"], 
    default="commoner"
)
df

,EMP_NO,NAME,SALARY,DEPARTMENT,rank
0,1.0,NaN,1000,IT,commoner
1,NaN,Bill,3000,IT,commoner
2,3.0,Charlie,2000,HR,commoner
3,4.0,Dragon,5000,IT,rich
4,5.0,Entropy,10000,NaN,rich
5,6.0,False,100,HR,commoner


## 4. 원 핫 인코딩

In [136]:
# DEPARTMENT 컬럼에 대해
pd.get_dummies(df, columns=["DEPARTMENT"])

,EMP_NO,NAME,SALARY,rank,DEPARTMENT_HR,DEPARTMENT_IT
0,1.0,NaN,1000,commoner,0,1
1,NaN,Bill,3000,commoner,0,1
2,3.0,Charlie,2000,commoner,1,0
3,4.0,Dragon,5000,rich,0,1
4,5.0,Entropy,10000,rich,0,0
5,6.0,False,100,commoner,1,0


## 5. 데이터 병합

In [156]:
# 데이터 조인
df2 = pd.DataFrame({
    "DEPARTMENT": ["HR", "IT", "CEO"],
    "WORKPLACE": ["R3", "R4", "R%"]
})
df.merge(df2, on=["DEPARTMENT"], how='inner')

# 데이터 필터링 후 인덱스 초기화
df[df["EMP_NO"] > 2].reset_index(drop=True)

# 데이터 프레임 행으로 합치기
df3 = pd.DataFrame({
    "WORKPLACE": ["R5", "R3", "R3", "R4", "R5", "R4"],
})
pd.concat([df, df3], axis=1)

,EMP_NO,NAME,SALARY,DEPARTMENT,rank,WORKPLACE
0,1.0,NaN,1000,IT,commoner,R5
1,NaN,Bill,3000,IT,commoner,R3
2,3.0,Charlie,2000,HR,commoner,R3
3,4.0,Dragon,5000,IT,rich,R4
4,5.0,Entropy,10000,NaN,rich,R5
5,6.0,False,100,HR,commoner,R4


## 6. 데이터 정렬 및 변환 

In [ ]:
# 정렬
df.sort_values(by=["NAME", "DEPARTMENT"])

# wide-form -> long-form 변환
df2 = pd.DataFrame({
    "YEAR": [2017, 2017],
    "MONTH": [1, 1],
    "DAY": [1, 2],
    "X0HR": [1, 1],
    "X1HR": [2, 2],
    "X2HR": [3, 3],
    "X3HR": [4, 4],
    "X4HR": [5, 5],
    "X5HR": [6, 6]
})
df2 = df2.melt(id_vars=["YEAR", "MONTH", "DAY"], var_name="HOUR", value_name="VALUE")

# long-form -> wide-form 변환 (안됨)
df3 = df2.pivot(index=["YEAR", "MONTH", "DAY"], columns=["var"], values="value")